# Reproduce R. H. Blackmore, et. al, Dalton Trans., 2020,49, 232-240 

This notebook demonstrates reproducing the XAS results of the paper with [Larch](https://xraypy.github.io/xraylarch/xafs/) using the corresponding published dataset (raw data). 

R. H. Blackmore, M. E. Rivas, T. Eralp Erden, T. Dung Tran, H. R. Marchbank, D. Ozkaya, M. Briceno de Gutierrez, A. Wagland, P. Collier, P. P. Wells (2018). **Understanding the mechanochemical synthesis of the perovskite LaMnO3 and its catalytic behaviour ** DOI: [10.1039/C9DT03590G](http://dx.doi.org/10.1039/C9DT03590G).

- **Corresponding authors**:  Emma K. Gibson, Kevin Morgan
- **E-mails**: 

The data used for reproducing the results was published in the University of Southampton Institutional Repository [doi:10.5258/SOTON/D1128](http://dx.doi.org/10.5258/SOTON/D1128)

For more details about Larch, see [Larch Website](https://xraypy.github.io/xraylarch/xafs/)